## Computer Assignment 1

AhmadReza Nopoush

id: 610301194

### Part 1: Initialization

#### importing libraries:

In [1]:
from game import Game
import time
import os
import re
import queue
import heapq

#### Loading maps:

In [2]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

#### Load a map:

In [3]:
print("This is an example of the game map:")
map = load_map(9)
game = Game(map)
game.display_map()

This is an example of the game map:
W	W	W	W	W	W	W	W	W	W	W	W
W	.	.	.	.	.	.	W	.	.	.	W
W	.	.	.	B2	.	.	W	.	G1	.	W
W	.	.	.	.	.	.	W	G4	P1	G2	W
W	.	.	.	.	.	.	W	.	G3	.	W
W	W	W	W	W	W	.	W	W	W	W	W
W	.	.	.	.	.	.	.	.	.	.	W
W	.	.	.	.	B4	.	.	.	.	.	W
W	.	.	H	B3	.	.	P1	B1	.	.	W
W	.	.	.	.	.	.	.	.	.	.	W
W	.	.	.	.	.	.	.	.	.	.	W
W	W	W	W	W	W	W	W	W	W	W	W


### Part 2: Question 1- 4

**Question 1:** 
What can be considered as the definition of a state in this problem? The most obvious definition is the map of the warehouse at any given moment. What is the problem with this definition? How can it be improved?



**Answer:** 
the problem of this definition is that a map includes redundant information (e.g., walls, portals and empty spaces that do not change). This leads to high memory usage and slower performance, because each time we should spend so much time on loading the map. Fortunately, we can represent the state using only the positions of the player and the boxes Instead of storing the entire map; because the only thing that changes in this game is Mike position and box locations. This reduces the state space significantly while retaining all necessary information.

---

**Question 2:** 
Based on your definition of a state, how would you define the actions?

**Answer:** 
as it can be observed from ``GUI`` file and the game Rules, Mike can move to the neighbor houses, which means that he can go **Up** or **Down** or **Left** or **Right**. so we define action as below:
- $ {{U, D, L, R }} $

-----

**Question 3:** 
Explain how to model the problem, including the definition of actions, transitions, and the goal state.



**Answer:** 

- States: Represented by the positions of the player and the boxes. I used ``tuple`` to work with it.

- Actions: The four possible moves (U, D, L, R).

- Transitions: Applying an action updates the player’s position and, if applicable, the box’s position. Portals are handled by teleporting the player or box to the corresponding portal exit. the ``Game`` library has an attribute named ``.apply_move()`` which can be used to see that if a move is valid and update the state if the move is valid.

- Goal State: We achieve when all boxes are on their respective goal positions. we can use the attribute ``is_game_won()`` to check if Mike did the job or not.


---

**Question 4:** 
How can this problem be improved, and how can the search space be optimized? Should we expand all possible states in the next step of the search algorithms? What are the consequences of this? What measures and considerations can be taken to improve the problem?



**Answer:** 

we can do optimization by:
1. Use heuristic functions to guide the search, which we implement in A*
2. Implement state pruning to avoid revisiting states. (for example using Visited Set)
3. Use efficient data structures (e.g., priority queues for A* or sets to store visited states).
4. Use iterative deepening (IDS) to balance memory usage and completeness.

we have no choice but expanding all possible in next step if the map is difficult to solve. for more simple maps, if we have a good heuristic function, which shows us a good path to achieve the goal, we can ignore many states and travers the path which heuristic function says. 


---

### Part 3: BFS

BFS produce optimal solution, because it explores all the states level by level. Its main advatage is that it guarantees the shortest path and its disadvantages is it uses high memory for large state spaces.

for implementing BFS, I define the Array of sets named ``Visited``, which each state that seen in BFS travers, will be stored in Visited. the reason that I implement in Array form is that when the size of set increases, the time complexity for search an element in set increases either, so I divide the Set into houses, each element of an array represents the position of the mike.

BFS algoritms are based on queues. so I design a queue that contains 3-tuple: Mike_Position, Box_Positins and the path. the algorithm is clear. and you can read the comments and follow it.

another reminder that I to call a specific state, I just change the box_locations and player_position. the reason is that the only things that changes in ``Game`` Object  is this two things.

In [4]:
def solver_bfs(game_map):

    State = Game(game_map)

    #a function that gives player position and return the index of it in Visited Array
    def hash_idx(player_position, height) -> int:
        return player_position[0] * height + player_position[1]
    
    #number of Visited States
    Count_Visited = 1
    
    #Queue contain (player position, box positions, path)
    Q = queue.Queue()
    Q.put((State.get_player_position(), tuple(State.get_box_locations()), []))

    map_height = State.height
    map_width = State.width

    #defining Visited Array
    Visited = [set() for i in range(map_height*map_width)]
    Visited[hash_idx(State.get_player_position(), map_height)].add(tuple(State.get_box_locations()))

    while not Q.empty():
        Player_Position, Box_Position, Path = Q.get()

        #Set the State
        State.set_player_position(Player_Position)
        State.set_box_positions(list(Box_Position))

        #terminate if we find a solution
        if State.is_game_won():
            return ''.join(Path), Count_Visited

        for direction in ['U', 'D', 'R', 'L']:
            result = State.apply_move(direction)

            #if the move is valid then:
            if result:
                New_Player_Position = State.get_player_position()
                New_Box_Position = tuple(State.get_box_locations())

                State.set_player_position(Player_Position)
                State.set_box_positions(list(Box_Position))

                index = hash_idx(New_Player_Position, map_height)

                #if the New State has not visited yet then:
                if New_Box_Position not in Visited[index]:

                    #Visit the State and add it to the queue
                    Visited[index].add(New_Box_Position)
                    Count_Visited += 1
                    Q.put((New_Player_Position, New_Box_Position, Path + [direction]))

    return None, Count_Visited

### Part 4: DFS

#### dfs : generate all solution

in this implementation, every solution to the game will be discover. That it means that the whole States of the game will be discoversd. but the question does not ask this from us.

In [5]:
def DFS(State: Game):
    Initial = (State.get_player_position(), tuple(State.get_box_locations()))
    Visited = set()
    Visited.add(Initial)
    solutions = [] 

    def dfs(path, visited: set): 
        State.set_player_position(Initial[0])
        State.set_box_positions(Initial[1])
        State.apply_moves(path)

        if State.is_game_won():
            print(''.join(path))
            solutions.append(''.join(path))
            return
        
        Current = (State.get_player_position(), tuple(State.get_box_locations()))
        
        for direction in ['U', 'D', 'L', 'R']:
            State.set_player_position(Current[0])
            State.set_box_positions(Current[1])
            result = State.apply_move(direction)
            if result:
                New_State = (State.get_player_position(), tuple(State.get_box_locations()))
                if New_State not in visited:
                    new_visited = set(visited)
                    new_visited.add(New_State)
                    dfs(path + [direction], new_visited)
    
    dfs([], Visited)
    return solutions

#### dfs :

in this implementation, the algorithm terminates when it finds first solution. there is two approches to implement DFS:
- use stack
- use recursive functions

because TA asked us to unvisit the visited State when we backtrack from a state, I choose second approches, because it is easier to handle the ``Visited`` set.

In [6]:
def solver_dfs(game_map):

    State  = Game(game_map)
    
    #Initial State, Start of the game
    Initial = (State.get_player_position(), tuple(State.get_box_locations()))
    Visited = set()
    Visited.add(Initial)

    Count_Visited = 1

    def dfs(path):
        nonlocal Count_Visited
        
        Current = (State.get_player_position(), tuple(State.get_box_locations()))
        
        if State.is_game_won():
            return ''.join(path)
        
        for direction in ['U', 'D', 'L', 'R']:
            State.set_player_position(Current[0])
            State.set_box_positions(list(Current[1]))
            
            if State.apply_move(direction):
                New_State = (State.get_player_position(), tuple(State.get_box_locations()))
                
                if New_State not in Visited:
                    Visited.add(New_State)
                    Count_Visited += 1

                    #Recursively run the algorithm
                    result = dfs(path + [direction])
                    if result:
                        return result
                    Visited.remove(New_State)
        
        return None

    return dfs([]),Count_Visited

### Part 5: IDS

IDS (Iterative Deepening Search) algorithm combines BFS and DFS by performing DFS with increasing depth limits. This algorithm guarantees the shortest path (optimal solution) with lower memory usage than BFS. However, it works for lower depths, but it is un efficient for memory in higher depth.

I have two problem to implement IDS. In one hand, I should consider that we do dfs in each depth, and when we backtrack from a state, we should unvisit that state (just like in dfs). In the other hand, if we just do dfs for each depth from begining, we do so much unneccery work, because when we want to perform a dfs for depth 14, we compute all depth 13 while we had computed the depth 13 in last iteration. so we should store each state in each depth level and use it for next iteration.

I implement IDS according to this as below:

In [20]:
def solver_ids(game_map):
    State = Game(game_map)
    
    def depth_limited_search(path, depth_limit, Visited):

        nonlocal Count_Visited

        Current = (State.get_player_position(), tuple(State.get_box_locations()))
        
        # Check if the current state is the goal state
        if State.is_game_won():
            return ''.join(path)
        
        # Stop if the depth limit is reached
        if depth_limit < 0:
            return None
        
        for direction in ['U', 'D', 'L', 'R']:
            # Reset to current state before applying the move
            State.set_player_position(Current[0])
            State.set_box_positions(list(Current[1]))
            
            # Apply the move
            if State.apply_move(direction):
                New_Player_Position = State.get_player_position()
                New_Box_Position = tuple(State.get_box_locations())
            
                new_state = (New_Player_Position, New_Box_Position)
                
                # Check if the new state has already been visited at this depth
                if new_state not in Visited:
                    Visited.add(new_state)  # Mark the new state as visited
                    Count_Visited += 1
                    
                    # Recursively search
                    result = depth_limited_search(path + [direction], depth_limit - 1, Visited)
                    if result:
                        return result
                    
                    # Backtrack: unvisit the state
                    Visited.remove(new_state)  
        
        return None

    depth_limit = 0
    Count_Visited = 1
    while depth_limit<State.height*State.width:
        
        Visited = set()
        initial_state = (State.get_player_position(), tuple(State.get_box_locations()))
        Visited.add(initial_state)
        
        result = depth_limited_search([], depth_limit, Visited)
        if result:
            return result,Count_Visited
        
        depth_limit += 1
    return None,Count_Visited

### Part 6: Question 5 

**Question 5:** 
Explain each of the implemented algorithms and their differences and advantages compared to each other. Which algorithms produce optimal solutions?



**Answer:** 

I explained about the implementation of this 3 algorithm before. Now let's compare!

BFS:

1. Explores all states level by level; so it guarantees yhe optimal solution.

2. Advantages: Guarantees the shortest path (optimal solution).

3. Disadvantages: High memory usage for large state spaces; because it should explore all states level by level, and the search space groath is exponantial!

DFS:

1. Explores as far as possible along each branch before backtracking.

2. Advantages: Low memory usage.

3. Disadvantages: Does not guarantee the shortest path and it can get stuck in infinite loops.

IDS:

1. Combines BFS and DFS by performing DFS with increasing depth limits.

2. Advantages: Guarantees the shortest path (optimal solution) with lower memory usage than BFS.

3. Disadvantages: Repeats work for lower depths, but this is offset by its memory efficiency.


**a.**
What is the problem with the DFS algorithm? Despite this, why is DFS still used?

**answer:**
DFS explores as far as possible along each branch before backtracking. As a result, it may find a solution that is not optimal. for big maps, for example map 8, it is not work properly, because in dfs, algorithm go straight forward to the leaf of search tree, and big maps have really huge search trees!
However, DFS only needs to store the current path, making it suitable for problems with large state spaces where memory is a constraint.and it can find a solution quickly if the solution is located deep in the search tree, even if it is not optimal. so because of this, dfs is used.


**b.**
Considering the advantages of BFS and DFS over each other, how does the IDS algorithm combine these two algorithms, and what problems does it aim to solve?

**answer:**

IDS use level by level search policy; which it is given by bfs. so because of that, IDS gives us optimal solution. it explore search tree just like dfs, so its memory usage for visited states is less than BFS. IDS try to be something between BFS and DFS


---

### Part 7: A* and Question 7


Implementation of A* Algorithm is similiar to BFS. The difference is that we use ``PriorityQueue`` which sorted by the value of cost function. cost function is the form of $$ f(n) = w*h(n) + g(n) $$ which $h(n) $ is heuristic function and $w$ is the weight.. the main problem of Implementing A* is that what heuristic should we design for the cost function? and How should be define $g(n)$? 

for the second, I define $ g(n) $ as the depth of the path that we have come so far. If $w = 0$ the performance of A* become just like BFS; because in BFS, States in queue are sorted based on the depth-level.

for the first one, one of the most simple answer is manhattan distance. This heuristic function computes the sum of Manhattan distances between each box and its goal. It does not consider portals or walls or the player's position. sadly it is not working for test case 9.

In [8]:
def manhattan_distance(State:Game):
        box_positions = State.get_box_locations()
        goal_positions = State.get_goal_locations()
        
        total_distance = 0
        for box, goal in zip(box_positions, goal_positions):
            total_distance += abs(box[0] - goal[0]) + abs(box[1] - goal[1])
        return total_distance

we can improve and modify the performance of manhattan distance by considering portals. the  manhattan distance is accurate when there is no walls and portals. but in challangable games we have both of it. by considering portal; we can have more accurate appromiximation of distance. ``heuristic_portal``, calculates the minimum distance between each box and its corresponding goal, considering the use of portals as shortcuts

In [9]:
def heuristic_portal(State:Game):
    portals_position = State.get_portal_locations()
    box_positions = State.get_box_locations()
    goal_positions = State.get_goal_locations()

    total_dist = 0

    for i in range(len(box_positions)): 
        box = box_positions[i]
        goal = goal_positions[i]

        min_dist = abs(box[0] - goal[0]) + abs(box[1] - goal[1])

        for portal_entry, portal_exit in portals_position:
            entry_to_exit_dist = abs(box[0] - portal_entry[0]) + abs(box[1] - portal_entry[1]) + abs(portal_exit[0] - goal[0]) + abs(portal_exit[1] - goal[1])
            exit_to_entry_dist = abs(box[0] - portal_exit[0]) + abs(box[1] - portal_exit[1]) + abs(portal_entry[0] - goal[0]) + abs(portal_entry[1] - goal[1])

            min_dist = min(min_dist, entry_to_exit_dist, exit_to_entry_dist)

        total_dist += min_dist

    return total_dist



**Question 7:** 
Explain the heuristics you introduced in the informed search section and check if they are admissible and consistant.




**Answer:** 

A heuristic is admissible if it never overestimates the true cost to reach the goal. and a heuristic is consistent if for every move, the estimated cost never exceeds the actual cost between states.

Manhattan Heuristic is Admissible because tt always underestimates the true cost since only horizontal or vertical move is valid. and also it is Consistent because The heuristic decreases at most by 1 for each valid move. the player at every state at least do one step that make him close to the goal.

``heuristic_portal`` is Admissible; because it tries to estimate the shortest possible path including portals to move boxesx, so it never overestimates. And it is Consistent because Moving a box reduces the heuristic by at most the actual cost of the move.



---

at the end, we implement weighted A* algorithm:
we set ``weight=4`` initially. because for map 9 ``weight < 3`` does not working.
Intuitively, increasing the weight makes the heuristic function more effective and decreasing the weight, makes A* Algorithm similar to the BFS, because $ g(n) $ factor in $f(n)$ is based on depth and BFS the elements in queues are sorted based on their depth.

In [10]:
def solver_astar(game_map, heuristic_func=heuristic_portal, weight=4):
    def hash_idx(player_position, height) -> int:
        return player_position[0] * height + player_position[1]

    State = Game(game_map)
    
    Count_Visited = 1
    
    Priority_Queue = []
    heapq.heappush(Priority_Queue, (0, State.get_player_position(), tuple(State.get_box_locations()), []))

    map_height = State.height
    map_width = State.width

    Visited = [set() for _ in range(map_height * map_width)]
    initial_index = hash_idx(State.get_player_position(), map_height)
    Visited[initial_index].add(tuple(State.get_box_locations()))

    while Priority_Queue:
        _, Player_Position, Box_Position, Path = heapq.heappop(Priority_Queue)

        State.set_player_position(Player_Position)
        State.set_box_positions(list(Box_Position))

        if State.is_game_won():
            return ''.join(Path), Count_Visited

        for direction in ['U', 'D', 'R', 'L']:
            result = State.apply_move(direction)

            if result:
                New_Player_Position = State.get_player_position()
                New_Box_Position = tuple(State.get_box_locations())

                index = hash_idx(New_Player_Position, map_height)
                if New_Box_Position not in Visited[index]:
                    Visited[index].add(New_Box_Position)
                    Count_Visited += 1

                    g = len(Path) + 1
                    h = heuristic_func(State)
                    f = g + weight*h
                    heapq.heappush(Priority_Queue, (f, New_Player_Position, New_Box_Position, Path + [direction]))

                State.set_player_position(Player_Position)
                State.set_box_positions(list(Box_Position))

    return None, Count_Visited

### Part 8: Questions  6 and 8

In [11]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar
} 

def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

#### Question 6:

Implement the mentioned algorithms and justify the answers to Question 5 with your results.

**BFS for all maps exept map 9**

In [12]:
def solve_all():
    for map in range(min(map_file_indices), 11):
        if map==9:
            continue
        solver = {"BFS": solver_bfs}
        for method in solver.keys():
            solve(map, method)

solve_all()

BFS took 0.0 seconds on map 1 and visited 47 states.
7 moves were used: UDDURLL
BFS took 0.0 seconds on map 2 and visited 26 states.
6 moves were used: LUDDUL
BFS took 0.0 seconds on map 3 and visited 130 states.
13 moves were used: ULDDUUUURDDDD
BFS took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
BFS took 0.07 seconds on map 5 and visited 7816 states.
15 moves were used: ULDDRDLLLUUURUL
BFS took 0.2 seconds on map 6 and visited 20823 states.
34 moves were used: UUUUURRRLLLLLLLDDDDDDDDDRDLRRRRRRR
BFS took 8.82 seconds on map 7 and visited 802173 states.
34 moves were used: RURRDDDDLDRUUUULLLRDRDRDDLLDLLUUDR
BFS took 0.12 seconds on map 8 and visited 12996 states.
17 moves were used: URURDDLDRRRRURDDR
BFS took 2.67 seconds on map 10 and visited 255601 states.
46 moves were used: RRRRRDRULURULLLULDRUUULDRDLDRRDRULURURDDRDLLLL


**DFS for maps 1 - 4**

In [13]:
def solve_all():
    for map in range(1, 5):
        if map==9:
            continue
        solver = {"DFS": solver_dfs}
        for method in solver.keys():
            solve(map, method)

solve_all()

DFS took 0.0 seconds on map 1 and visited 11 states.
7 moves were used: UDDULRR
DFS took 0.0 seconds on map 2 and visited 8 states.
6 moves were used: LUDDUL
DFS took 0.0 seconds on map 3 and visited 139 states.
33 moves were used: ULUURDULDDDUUURDDUULDDDDUUUURDDDD
DFS took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!


**IDS for maps 1 - 5**

In [21]:
def solve_all():
    for map in range(1, 6):
        if map==9:
            continue
        solver = {"IDS": solver_ids}
        for method in solver.keys():
            solve(map, method)

solve_all()

IDS took 0.0 seconds on map 1 and visited 61 states.
6 moves were used: LUDDUL
IDS took 0.0 seconds on map 2 and visited 58 states.
6 moves were used: LUDDUL
IDS took 0.01 seconds on map 3 and visited 1227 states.
13 moves were used: ULDDUUUURDDDD
IDS took 0.0 seconds on map 4 and visited 36 states.
No Solution Found!
IDS took 2.25 seconds on map 5 and visited 552593 states.
15 moves were used: ULDDRDLLLUUURUL


**Answer:**

As we see, the BFS is faster than IDS and DFS to find solution but has more memory usage than this two. 

#### Question 8:

Run the algorithm using all the heuristics you introduced and compare their results.

**Run A_astar with menhattan distance as heuristic**

In [15]:
for map in range(1, 11):
    if map==9:
        continue
    loadmap = load_map(map)

    start_time = time.time()
    moves, numof_visited_states = solver_astar(loadmap,manhattan_distance,1)
    end_time = time.time()
    print(f"A* with manhattan took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")

A* with manhattan took 0.0 seconds on map 1 and visited 46 states.
7 moves were used: DURLUDL
A* with manhattan took 0.0 seconds on map 2 and visited 24 states.
6 moves were used: LDUUDL
A* with manhattan took 0.0 seconds on map 3 and visited 106 states.
13 moves were used: ULDDUUUURDDDD
A* with manhattan took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
A* with manhattan took 0.01 seconds on map 5 and visited 881 states.
15 moves were used: LULDDRDLLUUURUL
A* with manhattan took 0.07 seconds on map 6 and visited 7168 states.
34 moves were used: RRDDDDDRLLLLLLLUUUUUUUUURULRRRRRRR
A* with manhattan took 0.6 seconds on map 7 and visited 56211 states.
34 moves were used: RURRDDDDLDRUUUULLLRDRDRDDLLDLLURLU
A* with manhattan took 0.0 seconds on map 8 and visited 522 states.
19 moves were used: URRRRRRRRRRRRRURDDR
A* with manhattan took 0.66 seconds on map 10 and visited 69148 states.
46 moves were used: RRRRRDRULURULLLULLDRUULDRDLDRRDRULURURDDRDLLLL


**Run weighted A_astar with menhattan distance as heuristic with weight = 3**

In [16]:
for map in range(1, 11):
    
    loadmap = load_map(map)

    start_time = time.time()
    moves, numof_visited_states = solver_astar(loadmap,manhattan_distance,3)
    end_time = time.time()
    print(f"A* with manhattan took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")

A* with manhattan took 0.0 seconds on map 1 and visited 23 states.
7 moves were used: UDLRRLD
A* with manhattan took 0.0 seconds on map 2 and visited 16 states.
6 moves were used: LUDLRD
A* with manhattan took 0.0 seconds on map 3 and visited 34 states.
13 moves were used: ULDDUUUURDDDD
A* with manhattan took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
A* with manhattan took 0.0 seconds on map 5 and visited 149 states.
15 moves were used: LULDLRDRDLLULUU
A* with manhattan took 0.02 seconds on map 6 and visited 2013 states.
34 moves were used: RRDDDDDRLLLLLLLUUUUUUUUURULRRRRRRR
A* with manhattan took 0.09 seconds on map 7 and visited 9384 states.
38 moves were used: RURRDLLLRRRDDDLDRUUUULLDRDRDDLLDLLUUDR
A* with manhattan took 0.0 seconds on map 8 and visited 76 states.
19 moves were used: URRRRRRRRRRRRRURDDR
A* with manhattan took 12.57 seconds on map 9 and visited 963629 states.
59 moves were used: RRLURRRDLURDLLULURURDURDRDLRRDLLDLULUUUUUULLLURRURDDDDDDLDR
A* with m

**Run A_astar with heuristic_portal as heuristic**

In [17]:
for map in range(1, 11):
    if map==9:
        continue
    
    loadmap = load_map(map)

    start_time = time.time()
    moves, numof_visited_states = solver_astar(loadmap,heuristic_portal,1)
    end_time = time.time()
    print(f"A* with heuristic_portal took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")

A* with heuristic_portal took 0.0 seconds on map 1 and visited 46 states.
7 moves were used: DURLUDL
A* with heuristic_portal took 0.0 seconds on map 2 and visited 24 states.
6 moves were used: LDUUDL
A* with heuristic_portal took 0.0 seconds on map 3 and visited 106 states.
13 moves were used: ULDDUUUURDDDD
A* with heuristic_portal took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
A* with heuristic_portal took 0.01 seconds on map 5 and visited 881 states.
15 moves were used: LULDDRDLLUUURUL
A* with heuristic_portal took 0.08 seconds on map 6 and visited 7168 states.
34 moves were used: RRDDDDDRLLLLLLLUUUUUUUUURULRRRRRRR
A* with heuristic_portal took 0.78 seconds on map 7 and visited 56211 states.
34 moves were used: RURRDDDDLDRUUUULLLRDRDRDDLLDLLURLU
A* with heuristic_portal took 0.01 seconds on map 8 and visited 739 states.
17 moves were used: URURDDLDRRRRURDDR
A* with heuristic_portal took 1.36 seconds on map 10 and visited 79898 states.
46 moves were used: RRRRRDRU

**Run weighted A_astar with heuristic_portal as heuristic with weight = 3**

In [18]:
for map in range(1, 11):
    
    loadmap = load_map(map)

    start_time = time.time()
    moves, numof_visited_states = solver_astar(loadmap,heuristic_portal,3)
    end_time = time.time()
    print(f"A* with heuristic_portal took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")

A* with heuristic_portal took 0.0 seconds on map 1 and visited 23 states.
7 moves were used: UDLRRLD
A* with heuristic_portal took 0.0 seconds on map 2 and visited 16 states.
6 moves were used: LUDLRD
A* with heuristic_portal took 0.0 seconds on map 3 and visited 34 states.
13 moves were used: ULDDUUUURDDDD
A* with heuristic_portal took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
A* with heuristic_portal took 0.0 seconds on map 5 and visited 149 states.
15 moves were used: LULDLRDRDLLULUU
A* with heuristic_portal took 0.03 seconds on map 6 and visited 2013 states.
34 moves were used: RRDDDDDRLLLLLLLUUUUUUUUURULRRRRRRR
A* with heuristic_portal took 0.14 seconds on map 7 and visited 9384 states.
38 moves were used: RURRDLLLRRRDDDLDRUUUULLDRDRDDLLDLLUUDR
A* with heuristic_portal took 0.0 seconds on map 8 and visited 367 states.
21 moves were used: URURDDDDLDRRRRLUURRDR
A* with heuristic_portal took 2.56 seconds on map 9 and visited 171519 states.
55 moves were used: RRLU

### Part 9: Question 9 and Results

we can conclude all the codes and algorithms above in this table:

| Algorithm      | Optimality      | Memory Usage     | Speed            | Advantages                                   | Disadvantages                              |
|----------------|-----------------|------------------|------------------|---------------------------------------------|-------------------------------------------|
| **BFS**        | Yes             | High             | Fast             | Guarantees shortest path                    | High memory usage for large state spaces  |
| **DFS**        | No              | Low              | Slow  | Low memory usage; quick for deep solutions | No guarantee of optimality; infinite loops |
| **IDS**        | Yes             | Low              | Moderate         | Combines optimality and memory efficiency   | Repeats work for lower depths             |
| **A***         | Yes (if heuristic is admissible) | Moderate | Fast (with good heuristic) | Efficient with good heuristic               | Requires a good heuristic; higher memory  |
| **Weighted A***| No (if w > 1)   | Moderate         | Very Fast        | Faster than A*                              | May produce suboptimal solutions          |

#### map 1:

| Algorithm         | Execution Time (s) | Solution (Output) | States Visited | Length of Path |
|-------------------|--------------------|-------------------|----------------|----------------|
| BFS               |      0.0           |   UDDURLL         |     47         |  7             | 
| DFS               |      0.0           |   UDDULRR         |     11         |  7             |
| IDS               |      0.0           |   LUDDUL          |     61         |  6             |
| A*, M. dist.      |      0.0           |   DURLUDL         |     46         |  7             |
| W. A*, M. dist.   |      0.0           |   UDLRRLD         |     23         |  7             |
| A*, h_p           |      0.0           |   DURLUDL         |     46         |  7             |
| W. A*, h_p        |      0.0           |   UDLRRLD         |     23         |  7             |

#### map 2:

| Algorithm         | Execution Time (s) | Solution (Output) | States Visited | Length of Path |
|-------------------|--------------------|-------------------|----------------|----------------|
| BFS               |      0.0           |   LUDDUL          |     26         |  6             | 
| DFS               |      0.0           |   LUDDUL          |     8          |  6             |
| IDS               |      0.0           |   LUDDUL          |     58         |  6             |
| A*, M. dist.      |      0.0           |   LDUUDL          |     24         |  6             |
| W. A*, M. dist.   |      0.0           |   LUDLRD          |     16         |  6             |
| A*, h_p           |      0.0           |   LDUUDL          |     24         |  6             |
| W. A*, h_p        |      0.0           |   LUDLRD          |     16         |  6             |

#### map 3:

| Algorithm         | Execution Time (s) | Solution (Output) | States Visited | Length of Path |
|-------------------|--------------------|-------------------|----------------|----------------|
| BFS               |      0.0           |   ULDDUUUURDDDD                           |     130         |  13             | 
| DFS               |      0.0           |   ULUURDULDDDUUURDDUULDDDDUUUURDDDD |     139          |  33             |
| IDS               |      0.01          |   ULDDUUUURDDDD                     |   1227         |  13             |
| A*, M. dist.      |      0.0           |   ULDDUUUURDDDD                     |    106         |  13             |
| W. A*, M. dist.   |      0.0           |   ULDDUUUURDDDD                     |     34         |  13             |
| A*, h_p           |      0.0           |   ULDDUUUURDDDD                     |    106         |  13             |
| W. A*, h_p        |      0.0           |   ULDDUUUURDDDD                     |     34         |  13             |

#### map 4:

| Algorithm         | Execution Time (s) | Solution (Output) | States Visited | Length of Path |
|-------------------|--------------------|-------------------|----------------|----------------|
| BFS               |      0.0           |   No          |     2         |  0             | 
| DFS               |      0.0           |   No          |     2          |  0             |
| IDS               |      0.0           |   No          |     3         |  0             |
| A*, M. dist.      |      0.0           |   No          |     2         |  0             |
| W. A*, M. dist.   |      0.0           |   No          |     2         |  0             |
| A*, h_p           |      0.0           |   No          |     2         |  0             |
| W. A*, h_p        |      0.0           |   No          |     2         |  0             |

#### map 5:

| Algorithm         | Execution Time (s) | Solution (Output) | States Visited | Length of Path |
|-------------------|--------------------|-------------------|----------------|----------------|
| BFS               |      0.07          |   ULDDRDLLLUUURUL          |     7816         |  15             | 
| IDS               |      2.25           |   ULDDRDLLLUUURUL          |     552593         |  15             |
| A*, M. dist.      |      0.01           |   LULDDRDLLUUURUL          |     881         |  15             |
| W. A*, M. dist.   |      0.0           |   LULDLRDRDLLULUU          |     149         |  15             |
| A*, h_p           |      0.01           |   LULDDRDLLUUURUL          |     881         |  15             |
| W. A*, h_p        |      0.0           |   LULDLRDRDLLULUU          |     149         |  15             |

#### map 6:

| Algorithm         | Execution Time (s) | Solution (Output) | States Visited | Length of Path |
|-------------------|--------------------|-------------------|----------------|----------------|
| BFS               |      0.2           |   UUUUURRRLLLLLLLDDDDDDDDDRDLRRRRRRR          |     20823         |  34             | |
| A*, M. dist.      |      0.07           |   RRDDDDDRLLLLLLLUUUUUUUUURULRRRRRRR          |     7168         |  34             |
| W. A*, M. dist.   |      0.02           |   RRDDDDDRLLLLLLLUUUUUUUUURULRRRRRRR          |     2013         |  34             |
| A*, h_p           |      0.08           |   RRDDDDDRLLLLLLLUUUUUUUUURULRRRRRRR          |     7168         |  34             |
| W. A*, h_p        |      0.03           |   RRDDDDDRLLLLLLLUUUUUUUUURULRRRRRRR          |     2013         |  34             |

#### map 7:

| Algorithm         | Execution Time (s) | Solution (Output) | States Visited | Length of Path |
|-------------------|--------------------|-------------------|----------------|----------------|
| BFS               |      8.81           |   RURRDDDDLDRUUUULLLRDRDRDDLLDLLUUDR          |     802173         |  34             | |
| A*, M. dist.      |      0.6           |   RURRDDDDLDRUUUULLLRDRDRDDLLDLLURLU          |     56211         |  34             |
| W. A*, M. dist.   |      0.09           |   RURRDLLLRRRDDDLDRUUUULLDRDRDDLLDLLUUDR          |     9384         |  38             |
| A*, h_p           |      0.78           |   RURRDDDDLDRUUUULLLRDRDRDDLLDLLURLU          |     56211         |  34             |
| W. A*, h_p        |      0.14           |   RURRDLLLRRRDDDLDRUUUULLDRDRDDLLDLLUUDR          |     9384         |  38             |

#### map 8:

| Algorithm         | Execution Time (s) | Solution (Output) | States Visited | Length of Path |
|-------------------|--------------------|-------------------|----------------|----------------|
| BFS               |      0.12           |   URURDDLDRRRRURDDR          |     12996         |  17             | 
| A*, M. dist.      |      0.0           |   URRRRRRRRRRRRRURDDR          |     552         |  19             |
| W. A*, M. dist.   |      0.0           |   URRRRRRRRRRRRRURDDR          |     76         |  19             |
| A*, h_p           |      0.01           |   URURDDLDRRRRURDDR          |     739         |  17             |
| W. A*, h_p        |      0.0           |   URURDDDDLDRRRRLUURRDR          |     367         |  21             |

#### map 9:

| Algorithm         | Execution Time (s) | Solution (Output) | States Visited | Length of Path |
|-------------------|--------------------|-------------------|----------------|----------------|
| W. A*, M. dist.   |      12.57           |   RRLURRRDLURDLLULURURDURDRDLRRDLLDLULUUUUUULLLURRURDDDDDDLDR          |     963629          |  59             |
| W. A*, h_p        |      2.56           |   RRLURRRRDRDLDLULULURURDURRDRDLLUULLUUULLLURRURDDDDDDLDR          |     171519         |  55             |

#### map 10

| Algorithm         | Execution Time (s) | Solution (Output) | States Visited | Length of Path |
|-------------------|--------------------|-------------------|----------------|----------------|
| BFS               |      2.67           |   RRRRRDRULURULLLULDRUUULDRDLDRRDRULURURDDRDLLLL          |     255601         |  46      |
| A*, M. dist.      |      0.66           |   RRRRRDRULURULLLULLDRUULDRDLDRRDRULURURDDRDLLLL          |     69148         |  46         |
| W. A*, M. dist.   |      0.2           |   RRRRRDRULURULLLULLDRUULDRDLDRRDRULURURDDRDLLLL          |     16715         |  46        |
| A*, h_p           |      1.36           |   RRRRRDRULURULLLULLDRUULDRDLDRRDRULURURDDRDLLLL          |     79898         |  46        |
| W. A*, h_p        |      0.04           |   RRRRRDRULURULLLULLDRUULDRDLDRRDRULURURDDRDLLLL          |     3438         |  46     |